### Test case LVV-T3155: Verify Engineering and Facility Database Availability

Demonstrate Engineering and Facilities Data are available for public access within **L1PublicT (24 hours)**.

#### Discussion:

This can be verified using the "ConsDB" (Consolidated database), which is a transformed version of telemetry from the Engineering Facilities Database (EFD), with additional tables and their columns populated by quantities derived from images immediately after they are obtained.

In particular, this test will use data and telemetry from LSSTCam observing in June 2025. All queries will be limited to the current incoming data stream, and will be executed while LSSTCam observing is actively being conducted.

#### Look up the current time and print it to the screen:

In [1]:
import datetime

# Note that Rubin Observatory reports quantities in TAI, but for
#   our present purposes, the 37-second difference between UTC and TAI
#   is unimportant.
time_now = datetime.datetime.now(datetime.UTC)
print(time_now)

2025-06-13 23:18:03.184344+00:00


The current UTC time is June 13, 2025, so we will select an observing date of June 13, 2025 (the day_obs is the local date in Chile on which the observing night began).

> Note that unfortunately the dome is closed on this night due to weather, so there is no on-sky observing. Thus all observations seen here are in-dome calibrations, which may not have the full information that on-sky data contain.

In [2]:
# Parameters to select a single observation date and the LSSTCam instrument:
day_obs = "2025-06-13"
instrument = "lsstcam"

Import packages, and set some variables for authentication to ConsDB:

In [3]:
import numpy as np
import pandas as pd
import requests
from IPython.display import Markdown, display, display_markdown
from astropy.table import Table
from lsst.rsp import get_tap_service

%matplotlib inline

Authenticate to and access the ConsDB:

In [4]:
cdb = get_tap_service("consdbtap")
print(cdb)
assert cdb is not None

display_markdown("Consolidated Database is accessible", raw=True)

TAPService(baseurl : 'https://usdf-rsp.slac.stanford.edu/api/consdbtap', description : 'None')


Consolidated Database is accessible

Query the ConsDB for data from today's date

In [5]:
day_obs_int = int(day_obs.replace("-", ""))
print(f'Date: {day_obs_int}')

# Query the visit table
visit_query1 = f"""
    SELECT * FROM cdb_{instrument}.visit1
    where day_obs <= {day_obs_int} AND day_obs > {day_obs_int-1}
"""

# Execute the query:
try:
    visits = cdb.search(visit_query1).to_table()
except requests.HTTPError or requests.JSONDecodeError:
    # Try twice
    visits = client.query(visit_query1).to_pandas()

# Print some stats to the screen:
if len(visits) > 0:
    print(f"\nRetrieved {len(visits)} visits from consdb")
    obj_vis = len(np.where(visits['img_type'] == "OBJECT")[0])
    print(f"{obj_vis} of these are object images")


Date: 20250613

Retrieved 133 visits from consdb
0 of these are object images


#### Print the list of columns in this table:

In [6]:
cols = [col for col in visits.columns]
print(cols)

['air_temp', 'airmass', 'altitude', 'altitude_end', 'altitude_start', 'azimuth', 'azimuth_end', 'azimuth_start', 'band', 'can_see_sky', 'controller', 'cur_index', 'dark_time', 'day_obs', 'dimm_seeing', 'emulated', 'exp_midpt', 'exp_midpt_mjd', 'exp_time', 'exposure_name', 'focus_z', 'group_id', 'humidity', 'img_type', 'max_index', 'obs_end', 'obs_end_mjd', 'obs_start', 'obs_start_mjd', 'observation_reason', 'physical_filter', 'pressure', 's_dec', 's_ra', 's_region', 'scheduler_note', 'science_program', 'seq_num', 'shut_time', 'simulated', 'sky_rotation', 'target_name', 'vignette', 'vignette_min', 'visit_id', 'wind_dir', 'wind_speed', 'zenith_distance', 'zenith_distance_end', 'zenith_distance_start']


#### Display statistics related to the date/time and duration of the observation, and the filter:

Note that this is calibration observing, so some columns that may be useful for on-sky observing may not be populated in this particular dataset.

In [7]:
obs_columns = ['visit_id', 'day_obs', 'obs_start', 'obs_end', 'img_type', 'exp_time', 'shut_time',
               'dark_time', 'physical_filter', 'band']

obs_dict = {}

for col in obs_columns:
    obs_dict[col] = visits[10:30][col]

tab = Table(obs_dict)
tab

visit_id,day_obs,obs_start,obs_end,img_type,exp_time,shut_time,dark_time,physical_filter,band
,,,,,s,s,s,,
int64,int32,object,object,object,float32,float32,float32,object,object
2025061300011,20250613,2025-06-13T13:49:47.877,2025-06-13T13:49:47.891,bias,0.0,0.0,0.01395607,i_39,i
2025061300012,20250613,2025-06-13T13:59:50.662,2025-06-13T13:59:50.676,bias,0.0,0.0,0.014225483,i_39,i
2025061300013,20250613,2025-06-13T14:09:53.607,2025-06-13T14:09:53.619,bias,0.0,0.0,0.012476921,i_39,i
2025061300014,20250613,2025-06-13T14:19:56.657,2025-06-13T14:19:56.669,bias,0.0,0.0,0.011762857,i_39,i
2025061300015,20250613,2025-06-13T14:29:59.588,2025-06-13T14:29:59.601,bias,0.0,0.0,0.013347149,i_39,i
2025061300016,20250613,2025-06-13T14:40:02.626,2025-06-13T14:40:02.640,bias,0.0,0.0,0.013904095,i_39,i
2025061300017,20250613,2025-06-13T14:50:05.652,2025-06-13T14:50:05.667,bias,0.0,0.0,0.014282703,i_39,i
2025061300018,20250613,2025-06-13T15:00:08.292,2025-06-13T15:00:08.306,bias,0.0,0.0,0.014017582,i_39,i


Notice that the times printed (in "obs_start") are much less than 24 hours before the current time shown above. We have thus demonstrated that the databases are populated within L1PublicT = 24 hours.

#### Display quantities related the dome and telescope pointing and orientation:

In [8]:
tel_columns = ['visit_id', 'sky_rotation', 'azimuth_start', 'altitude_start']

tel_dict = {}

for col in tel_columns:
    tel_dict[col] = visits[:30][col]

tab = Table(tel_dict)
tab

visit_id,sky_rotation,azimuth_start,altitude_start
,deg,deg,deg
int64,float64,float32,float32
2025061300001,57.984363316558245,328.00635,90.0
2025061300002,57.977245203602095,328.00635,90.0
2025061300003,57.9702098181699,328.00635,90.0
2025061300004,57.96326127796089,328.00635,90.0
2025061300005,57.95640482971007,328.00635,90.0
2025061300006,57.94964856391615,328.00635,90.0
2025061300007,57.943109612533526,328.00635,90.0
2025061300008,57.93672371684217,328.00635,90.0


#### Display quantities related to the airmass, zenith distance, and environmental conditions of observations:

In [9]:
env_columns1 = ['visit_id', 'zenith_distance_start', 'zenith_distance_end', 'zenith_distance', 'airmass']

env_dict1 = {}

for col in env_columns1:
    env_dict1[col] = visits[-30:][col]

tab = Table(env_dict1)
tab

visit_id,zenith_distance_start,zenith_distance_end,zenith_distance,airmass
,deg,deg,deg,
int64,float32,float32,float32,float32
2025061300104,68.0,68.0,--,--
2025061300105,68.0,68.0,--,--
2025061300106,68.0,68.0,--,--
2025061300107,68.0,68.0,--,--
2025061300108,68.0,68.0,--,--
2025061300109,68.0,68.0,--,--
2025061300110,68.0,68.0,--,--
2025061300111,68.0,68.0,--,--


In [10]:
env_columns2 = ['visit_id', 'air_temp', 'pressure', 'humidity', 'wind_speed', 'wind_dir',
                'dimm_seeing', 'focus_z']

env_dict2 = {}

for col in env_columns2:
    env_dict2[col] = visits[10:15][col]

tab = Table(env_dict2)
tab

visit_id,air_temp,pressure,humidity,wind_speed,wind_dir,dimm_seeing,focus_z
,,Pa,,m / s,deg,arcsec,
int64,float32,float32,float32,float32,float32,float32,float32
2025061300011,-0.675,73790.0,100.075,0.0,25.215,--,0.0
2025061300012,-0.7,73835.0,100.075,0.0,25.21,--,0.0
2025061300013,-0.65,73850.0,100.1,0.0,25.205,--,0.0
2025061300014,-0.7125,73850.0,100.075,0.0,25.205,--,0.0
2025061300015,-0.575,73850.0,100.1,0.0,25.21,--,0.0


In [11]:
# Query the ccdexposure table
ccdexp_query = f"""
    SELECT * FROM cdb_{instrument}.ccdexposure
    where day_obs <= {day_obs_int} AND day_obs > {day_obs_int-3}
"""

# Execute the query:
try:
    ccdexp = cdb.search(ccdexp_query).to_table()
except requests.HTTPError or requests.JSONDecodeError:
    # Try twice
    ccdexp = cdb.search(ccdexp_query).to_table()


In [12]:
ccdexp[-5:]

ccdexposure_id,day_obs,detector,exposure_id,s_region,seq_num
int64,int32,int32,int64,object,int32
47328560572,20250613,188,2025061300133,,133
47328560585,20250613,201,2025061300133,,133
47328560586,20250613,202,2025061300133,,133
47328560587,20250613,203,2025061300133,,133
47328560588,20250613,204,2025061300133,,133


## Results

We see that the ConsDB contains information taken directly from image headers, transformed data from the EFD, and derived data based on image processing. As required, these data include information about each exposure, include the telescope and instrument configuration, telemetry from the telescope, environmental and pointing information, and details about the camera.

We have demonstrated that the ConsDB tables were populated in less than L1PublicT=24 hours, as required.

The result of this test is **PASS**.